In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from mw_api_diff_utils import *
import pandas as pd
from db_utils import query_hive_ssh

# Talk Page Diff Sampling

In this notebook we will be taking samples from the full set of user talk page diffs for annotation.

## Bot Filtering

We don't want to include messages from bots in our sample.

### Resources
- [overview](https://www.mediawiki.org/wiki/Manual:Bots)
- [users in bot group](https://en.wikipedia.org/w/index.php?title=Special:ListUsers/bot&offset=BrokenAnchorBot&group=bot)
- [user group table docs](https://www.mediawiki.org/wiki/Manual:User_groups_table)
- [recent chnages table docs](https://www.mediawiki.org/wiki/Manual:Recentchanges_table)

### Summary
There is no great way to determine bots. Here are 3 strategies:

1. The `recentchnages` table contains 30 days of data and tells you which revisions where made by bots. This is the most reliable data, but there may be a seasonal trend in harassment that biases the sample.

2. Check if the user is in the bot group in the `user group` table. Drawbacks are:
    - Group membership can change over time. There are many bot-flagged edits by users that are no longer in a user group providing the bot right. Likewise there are many edits not bot-flagged by users that now have the bot right (which they may or may not use for each edit).
    - Not all bots are a member of this group (there are other groups that provide this right, sysop, for example).
3. Remove users with 'bot|Bot|BOT' in the user_name. Almost all bots in the 'bot' user group fit this pattern, but there will be many false positives. It seems unlikely that these false positives result in biased sample ...

For now I will go with option 3.

## Data Cleaning

For now, we basically just strip markup and remove duplicate diffs.

##### Steps
1. remove dupilicate rev_ids
3. remove duplicate diffs
4. clean diffs
     - ignore diffs with no content added
     - replace 'NEWLINE' with '\n'
     - remove comments from 'The Wikipedia Adventure'
     - strip mw markup

In [20]:
def clean(df):
    df.rename(columns = {'insertion': 'diff'}, inplace = True)
    df.dropna(subset = ['diff'], inplace = True)
    df['clean_diff'] = df['diff']
    df['clean_diff'] = df['clean_diff'].apply(lambda x: x.replace('NEWLINE', '\n'))
    df['clean_diff'] = df['clean_diff'].apply(strip_mw)
    try:
        del df['rank']
    except:
        pass
    df.drop_duplicates(subset = ['rev_id'], inplace = True)
    df.drop_duplicates(subset = ['diff'], inplace = True)
    df.dropna(subset = ['clean_diff'], inplace = True)
    df = df[df['clean_diff'] != '']
    return df

# Post Based Random Sample
Consider comments made since min_timestamp. Take n random comments from non-bot users.

In [3]:
n = 100000
min_timestamp = '2014-03-01T00:00:00Z'

In [4]:
query = """
SELECT 
    diffs.*
FROM
    enwiki.user_talk_diff diffs
WHERE
    rev_timestamp > '%(min_timestamp)s'
    AND user_text != 'MediaWiki message delivery'
    AND user_text != 'Maintenance script'
    AND user_text NOT RLIKE 'bot|Bot|BOT'
ORDER BY RAND()
LIMIT %(n)d
"""

params = {
    'n': n,
    'min_timestamp': min_timestamp
    }

post_sample_df = query_hive_ssh(query % params, 'post_sample.tsv', priority = True)
post_sample_df.columns = [c.split('.')[1] for c in post_sample_df.columns]

In [ ]:
clean_post_sample_df = clean(post_sample_df)
for k in [1, 10, 100]:
    clean_post_sample_df[:(k*1000)].to_csv('../data/%dk_post_sample.tsv' % k, sep = '\t')

# User Based Random Sample

Consider comments made since min_timestamp. Take up to k random comments from each of n random non-bot users

In [5]:
k = 5
n = 10000
min_timestamp = '2014-03-01T00:00:00Z'

In [6]:
query = """
SELECT 
    revisions.*
FROM
    (SELECT
        a.*
        FROM
        (
        SELECT
            d.*,
            RANK() OVER (PARTITION BY d.user_id ORDER BY RAND()) as rank
        FROM
            enwiki.user_talk_diff d
        WHERE 
            rev_timestamp > '%(min_timestamp)s'
        ) a
    WHERE rank <= %(k)d
    )
    revisions

JOIN
    (
    SELECT
        a.*
    FROM (
        SELECT
            user_id,
            RAND() as key
        FROM
            enwiki.user_talk_diff d
        WHERE 
            rev_timestamp > '%(min_timestamp)s'
            AND user_text != 'MediaWiki message delivery'
            AND user_text != 'Maintenance script'
            AND user_text NOT RLIKE 'bot|Bot|BOT'
        GROUP BY user_id
        ) a
    ORDER BY key
    LIMIT %(n)d
    ) users
ON
    revisions.user_id = users.user_id
"""

params = {
    'k': k,
    'n': n,
    'min_timestamp': min_timestamp
    }


user_sample_df = query_hive_ssh(query % params, 'random_sample.tsv', priority = True)
user_sample_df.columns = [c.split('.')[1] for c in user_sample_df.columns]

In [9]:
clean_user_sample_df = clean(user_sample_df)
clean_post_sample_df.to_csv('../data/%dk_users_%d_per_user_sample.tsv' % (int(n / 1000), k), sep = '\t')

# Blocked Users
For n random user block events that occured after min_timestamp, take up to the k most recent comments before being blocked for harassment or personal attacks.

In [7]:
k = 25
n = 10000
min_timestamp = '2010-03-01T00:00:00Z'

In [8]:
query = """

SELECT
    revisions.*
FROM
    (
    SELECT 
        diffs.*,
        RANK() OVER (PARTITION BY events.key, events.user_text ORDER BY diffs.rev_timestamp DESC) as rank
    FROM
        (
        SELECT
            a.*
        FROM
            (
            SELECT 
              log_title as user_text,
              log_timestamp,
              RAND() as key
            FROM enwiki.logging 
            WHERE
              log_type = 'block'
              AND log_action = 'block'
              AND log_comment RLIKE 'harassment|personal attack'
              AND log_timestamp > '%(min_timestamp)s'
            ) a
        ORDER BY key
        LIMIT %(n)d
        ) events
    JOIN
        enwiki.user_talk_diff diffs
    ON
        diffs.user_text = events.user_text
    WHERE
        diffs.rev_timestamp <= events.log_timestamp
    ) revisions
WHERE
    rank <= %(k)d
"""

params = {
    'k': k,
    'n': n,
    'min_timestamp': min_timestamp
    }


blocked_user_df = query_hive_ssh(query % params, 'random_sample.tsv', priority = True)
blocked_user_df.columns = [c.split('.')[1] for c in blocked_user_df.columns]

In [10]:
clean_blocked_user_df = clean(blocked_user_df)
clean_blocked_user_df.to_csv('../data/%dk_blocked_users_%d_per_user_sample.tsv' % (int(n / 1000), k), sep = '\t')